In [4]:
library(tidyverse)
library(repr)
library(digest)
library(gridExtra)
library(cowplot)
library(dplyr)
library(tidymodels)
library(GGally)
library(splines)

In [20]:
setwd("/Users/matthewgillies/FantasyPreds2023")
metrics <- read_csv("data/RawMetrics.csv")
head(metrics)
colnames(metrics)

New names:
• `` -> `...22`
Rows: 550 Columns: 22
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): last_name, first_name
dbl (19): player_id, year, player_age, xslg, xwoba, xobp, exit_velocity_avg,...
lgl  (1): ...22

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


last_name,first_name,player_id,year,player_age,xslg,xwoba,xobp,exit_velocity_avg,launch_angle_avg,⋯,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,sprint_speed,...22
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
Cabrera,Miguel,408234,2021,38,0.420,0.315,0.301,91.4,10.0,⋯,8.5,49.0,69.1,30.2,49.0,25.9,21.5,3.6,23.9,NA
Cruz Jr.,Nelson,443558,2021,40,0.512,0.363,0.341,92.9,11.8,⋯,8.1,52.5,73.8,31.0,43.4,25.5,22.0,9.1,26.0,NA
Peralta,David,444482,2021,33,0.345,0.293,0.313,89.3,5.3,⋯,4.5,41.8,66.3,27.4,54.9,15.6,23.2,6.3,27.6,NA
Blackmon,Charlie,453568,2021,34,0.450,0.359,0.369,87.6,10.2,⋯,3.5,38.5,67.9,26.2,47.4,22.3,24.4,5.9,27.7,NA
McCutchen,Andrew,457705,2021,34,0.431,0.344,0.352,88.8,14.6,⋯,8.4,39.5,63.7,19.1,40.9,30.5,21.6,7.0,28.7,NA
Turner,Justin,457759,2021,36,0.470,0.359,0.358,90.9,19.2,⋯,8.6,42.4,66.9,22.7,35.4,33.1,24.5,7.0,25.5,NA


[1] "last_name"            "first_name"           "player_id"           
 [4] "year"                 "player_age"           "xslg"                
 [7] "xwoba"                "xobp"                 "exit_velocity_avg"   
[10] "launch_angle_avg"     "sweet_spot_percent"   "barrel_batted_rate"  
[13] "solidcontact_percent" "hard_hit_percent"     "z_swing_percent"     
[16] "oz_swing_percent"     "groundballs_percent"  "flyballs_percent"    
[19] "linedrives_percent"   "popups_percent"       "sprint_speed"        
[22] "...22"

In [28]:
selected <- metrics %>% select(-"player_id", -"...22")
head(selected)

last_name,first_name,year,player_age,xslg,xwoba,xobp,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel_batted_rate,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,sprint_speed
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Cabrera,Miguel,2021,38,0.420,0.315,0.301,91.4,10.0,32.8,8.3,8.5,49.0,69.1,30.2,49.0,25.9,21.5,3.6,23.9
Cruz Jr.,Nelson,2021,40,0.512,0.363,0.341,92.9,11.8,31.8,13.6,8.1,52.5,73.8,31.0,43.4,25.5,22.0,9.1,26.0
Peralta,David,2021,33,0.345,0.293,0.313,89.3,5.3,27.2,5.0,4.5,41.8,66.3,27.4,54.9,15.6,23.2,6.3,27.6
Blackmon,Charlie,2021,34,0.450,0.359,0.369,87.6,10.2,32.6,7.0,3.5,38.5,67.9,26.2,47.4,22.3,24.4,5.9,27.7
McCutchen,Andrew,2021,34,0.431,0.344,0.352,88.8,14.6,33.6,9.0,8.4,39.5,63.7,19.1,40.9,30.5,21.6,7.0,28.7
Turner,Justin,2021,36,0.470,0.359,0.358,90.9,19.2,36.5,7.9,8.6,42.4,66.9,22.7,35.4,33.1,24.5,7.0,25.5


In [29]:
names <- selected %>% unite("player", last_name:first_name, sep = ", ", remove = T) %>% mutate(year = year + 1)
head(names)

player,year,player_age,xslg,xwoba,xobp,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel_batted_rate,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,sprint_speed
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Cabrera, Miguel",2022,38,0.420,0.315,0.301,91.4,10.0,32.8,8.3,8.5,49.0,69.1,30.2,49.0,25.9,21.5,3.6,23.9
"Cruz Jr., Nelson",2022,40,0.512,0.363,0.341,92.9,11.8,31.8,13.6,8.1,52.5,73.8,31.0,43.4,25.5,22.0,9.1,26.0
"Peralta, David",2022,33,0.345,0.293,0.313,89.3,5.3,27.2,5.0,4.5,41.8,66.3,27.4,54.9,15.6,23.2,6.3,27.6
"Blackmon, Charlie",2022,34,0.450,0.359,0.369,87.6,10.2,32.6,7.0,3.5,38.5,67.9,26.2,47.4,22.3,24.4,5.9,27.7
"McCutchen, Andrew",2022,34,0.431,0.344,0.352,88.8,14.6,33.6,9.0,8.4,39.5,63.7,19.1,40.9,30.5,21.6,7.0,28.7
"Turner, Justin",2022,36,0.470,0.359,0.358,90.9,19.2,36.5,7.9,8.6,42.4,66.9,22.7,35.4,33.1,24.5,7.0,25.5


In [30]:
fpoints <- read_csv("data/fpoints_data.csv")
head(fpoints)

Rows: 539 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): player
dbl (10): year, K, BB, RBI, TB, SB, G, R, Fpoints, fpoints_g

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


player,year,K,BB,RBI,TB,SB,G,R,Fpoints,fpoints_g
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Soto, Juan",2020,28,41,37,107,5,47,39,201,4.276596
"Freeman, Freddie",2020,37,45,53,137,2,60,51,251,4.183333
"Yelich, Christian",2019,118,80,97,328,30,130,100,517,3.976923
"Rendon, Anthony",2019,86,80,126,326,5,146,117,568,3.890411
"Judge, Aaron",2022,175,111,131,391,16,157,133,607,3.866242
"Trout, Mike",2019,120,110,104,303,11,134,110,518,3.865672


In [31]:
fpoints_main <- fpoints %>% select("player", "year", "fpoints_g")
head(fpoints_main)

player,year,fpoints_g
<chr>,<dbl>,<dbl>
"Soto, Juan",2020,4.276596
"Freeman, Freddie",2020,4.183333
"Yelich, Christian",2019,3.976923
"Rendon, Anthony",2019,3.890411
"Judge, Aaron",2022,3.866242
"Trout, Mike",2019,3.865672


In [32]:
colnames(names)

[1] "player"               "year"                 "player_age"          
 [4] "xslg"                 "xwoba"                "xobp"                
 [7] "exit_velocity_avg"    "launch_angle_avg"     "sweet_spot_percent"  
[10] "barrel_batted_rate"   "solidcontact_percent" "hard_hit_percent"    
[13] "z_swing_percent"      "oz_swing_percent"     "groundballs_percent" 
[16] "flyballs_percent"     "linedrives_percent"   "popups_percent"      
[19] "sprint_speed"

In [33]:
full_data <- right_join(names, fpoints_main, by = c("player", "year"))
head(full_data)

player,year,player_age,xslg,xwoba,xobp,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel_batted_rate,solidcontact_percent,hard_hit_percent,z_swing_percent,oz_swing_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent,sprint_speed,fpoints_g
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Cruz Jr., Nelson",2022,40,0.512,0.363,0.341,92.9,11.8,31.8,13.6,8.1,52.5,73.8,31.0,43.4,25.5,22.0,9.1,26.0,1.604839
"Blackmon, Charlie",2022,34,0.450,0.359,0.369,87.6,10.2,32.6,7.0,3.5,38.5,67.9,26.2,47.4,22.3,24.4,5.9,27.7,2.125926
"McCutchen, Andrew",2022,34,0.431,0.344,0.352,88.8,14.6,33.6,9.0,8.4,39.5,63.7,19.1,40.9,30.5,21.6,7.0,28.7,2.044776
"Turner, Justin",2022,36,0.470,0.359,0.358,90.9,19.2,36.5,7.9,8.6,42.4,66.9,22.7,35.4,33.1,24.5,7.0,25.5,2.429688
"Andrus, Elvis",2022,32,0.375,0.308,0.324,87.8,8.1,34.0,2.1,5.9,34.8,65.9,31.9,47.0,17.5,28.4,7.1,26.7,2.046980
"Santana, Carlos",2022,35,0.421,0.335,0.345,89.9,12.9,28.0,6.8,7.1,41.9,66.9,22.6,45.3,26.5,19.2,8.8,24.9,1.961832


In [34]:
write_csv(full_data, "data/full_data.csv")